In [7]:
import pandas as pd
import json
from pathlib import Path

RAW_DATA_PATH = Path(r"C:\Users\DELL\Desktop\GPP_task\GPP_Task3\data\online_retail_II.xlsx")


In [8]:
import pandas as pd
print(pd.__version__)


2.3.3


In [10]:
import pandas as pd
import json
from pathlib import Path

RAW_DATA_PATH = Path(r"C:\Users\DELL\Desktop\GPP_task\ecommerce-churn-prediction\data\raw\online_retail_II.xlsx")
OUTPUT_JSON = Path("data/raw/data_quality_summary.json")

print("Excel exists:", RAW_DATA_PATH.exists())

Excel exists: True


In [12]:

import pandas as pd
import json
from pathlib import Path

RAW_DATA_PATH = Path(r"C:\Users\DELL\Desktop\GPP_task\ecommerce-churn-prediction\data\raw\online_retail_II.xlsx")
OUTPUT_JSON = Path("data/raw/data_quality_summary.json")

SHEET_NAME = "Year 2010-2011"  

raw_df = pd.read_excel(RAW_DATA_PATH, sheet_name=SHEET_NAME)
raw_df["financial_year"] = SHEET_NAME

raw_df.shape


(541910, 9)

In [13]:
raw_df.head()
raw_df.info()
raw_df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Invoice         541910 non-null  object        
 1   StockCode       541910 non-null  object        
 2   Description     540456 non-null  object        
 3   Quantity        541910 non-null  int64         
 4   InvoiceDate     541910 non-null  datetime64[ns]
 5   Price           541910 non-null  float64       
 6   Customer ID     406830 non-null  float64       
 7   Country         541910 non-null  object        
 8   financial_year  541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 37.2+ MB


Invoice                0
StockCode              0
Description         1454
Quantity               0
InvoiceDate            0
Price                  0
Customer ID       135080
Country                0
financial_year         0
dtype: int64

In [17]:
rows_before = raw_df.shape[0]

clean_df = raw_df.dropna(subset=["Customer ID"])

rows_after = clean_df.shape[0]

retention = (rows_after / rows_before) * 100

rows_before, rows_after, retention


(541910, 406830, 75.07335166356037)

In [21]:
clean_df["Invoice"].astype(str).str.startswith("C").sum()


np.int64(8905)

In [23]:
clean_df = clean_df[~clean_df["Invoice"].astype(str).str.startswith("C")]


In [24]:
rows_after_cancel = clean_df.shape[0]
retention_after_cancel = (rows_after_cancel / rows_before) * 100
rows_after_cancel, retention_after_cancel


(397925, 73.43008986732114)

In [25]:
(clean_df["Quantity"] <= 0).sum()


np.int64(0)

In [26]:
clean_df.duplicated().sum()


np.int64(5192)

In [27]:
clean_df = clean_df.drop_duplicates()

In [28]:
rows_after_dup = clean_df.shape[0]
final_retention = (rows_after_dup / rows_before) * 100
rows_after_dup, final_retention


(392733, 72.4719971951062)

In [29]:
reference_date = clean_df["InvoiceDate"].max() + pd.Timedelta(days=1)
reference_date


Timestamp('2011-12-10 12:50:00')

### RFM stands for:
- Recency	-Days since last purchase
- Frequency	-Number of purchases
- Monetary	-Total spend

In [39]:
rfm = clean_df.groupby("Customer ID").agg({
    "InvoiceDate": lambda x: (reference_date - x.max()).days,
    "Invoice": "nunique",
    "Price": "sum"
}).reset_index()

rfm.columns = ["Customer ID", "Recency", "Frequency", "Monetary"]

rfm


,Customer ID,Recency,Frequency,Monetary
0,12346.0,326,1,1.04
1,12347.0,2,7,481.21
2,12348.0,75,4,178.71
3,12349.0,19,1,605.10
4,12350.0,310,1,65.30
...,...,...,...,...
4334,18280.0,278,1,47.65
4335,18281.0,181,1,39.36
4336,18282.0,8,2,62.39
4337,18283.0,4,16,1174.33


In [42]:
CHURN_THRESHOLD = 90

rfm["Churn"] = (rfm["Recency"] > CHURN_THRESHOLD).astype(int)

rfm["Churn"].value_counts()


Churn
0    2890
1    1449
Name: count, dtype: int64

In [43]:
churned_customers = rfm["Churn"].sum()
total_customers = rfm.shape[0]

churn_rate = (churned_customers / total_customers) * 100
churn_rate


np.float64(33.39479142659599)

In [40]:
rfm.describe()


,Customer ID,Recency,Frequency,Monetary
count,4339.000000,4339.000000,4339.000000,4339.000000
mean,15299.936852,92.518322,4.271952,282.908362
std,1721.889758,100.009747,7.705493,875.427416
min,12346.000000,1.000000,1.000000,0.000000
25%,13812.500000,18.000000,1.000000,51.615000
50%,15299.000000,51.000000,2.000000,125.450000
75%,16778.500000,142.000000,5.000000,289.160000
max,18287.000000,374.000000,210.000000,33332.450000


- avg_quantity_per_order -	Average units bought per transaction
- max_quantity - Largest single-order quantity
- min_quantity - Smallest purchase quantity
- std_quantity - Purchase variability
- total_items_purchased - Total items bought
- unique_products -	Product diversity
- unique_invoices -	Purchase frequency proxy

In [46]:
purchase_features = clean_df.groupby("Customer ID").agg(
    avg_quantity_per_order=("Quantity", "mean"),
    max_quantity=("Quantity", "max"),
    min_quantity=("Quantity", "min"),
    std_quantity=("Quantity", "std"),
    total_items_purchased=("Quantity", "sum"),
    unique_products=("StockCode", "nunique"),
    unique_invoices=("Invoice", "nunique")
).reset_index()

purchase_features.head()


,Customer ID,avg_quantity_per_order,max_quantity,min_quantity,std_quantity,total_items_purchased,unique_products,unique_invoices
0,12346.0,74215.000000,74215,74215,NaN,74215,1,1
1,12347.0,13.505495,240,2,18.856172,2458,103,7
2,12348.0,75.516129,144,1,51.091990,2341,22,4
3,12349.0,8.643836,36,1,6.982856,631,73,1
4,12350.0,11.588235,24,1,4.345383,197,17,1


In [49]:
purchase_features.isnull().sum()
purchase_features["std_quantity"] = purchase_features["std_quantity"].fillna(0)
purchase_features.isnull().sum()

Customer ID               0
avg_quantity_per_order    0
max_quantity              0
min_quantity              0
std_quantity              0
total_items_purchased     0
unique_products           0
unique_invoices           0
dtype: int64

In [51]:
rfm = rfm.merge(purchase_features, on="Customer ID", how="left")

In [52]:
rfm.shape
rfm.head()

,Customer ID,Recency,Frequency,Monetary,Churn,avg_quantity_per_order,max_quantity,min_quantity,std_quantity,total_items_purchased,unique_products,unique_invoices
0,12346.0,326,1,1.04,1,74215.000000,74215,74215,0.000000,74215,1,1
1,12347.0,2,7,481.21,0,13.505495,240,2,18.856172,2458,103,7
2,12348.0,75,4,178.71,0,75.516129,144,1,51.091990,2341,22,4
3,12349.0,19,1,605.10,0,8.643836,36,1,6.982856,631,73,1
4,12350.0,310,1,65.30,1,11.588235,24,1,4.345383,197,17,1


- total_revenue	- Lifetime value so far
- avg_order_value -	Average spend per transaction
- max_order_value -	Highest single purchase
- min_order_value - Smallest purchase
- std_order_value -	Spend variability
- revenue_per_item - Price sensitivity

In [55]:
monetary_features = clean_df.groupby("Customer ID").agg(
    total_revenue=("Price", "sum"),
    avg_order_value=("Price", "mean"),
    max_order_value=("Price", "max"),
    min_order_value=("Price", "min"),
    std_order_value=("Price", "std"),
    total_items=("Quantity", "sum")
).reset_index()

# Revenue per item
monetary_features["revenue_per_item"] = (
    monetary_features["total_revenue"] / monetary_features["total_items"]
)

# Handle NaN (customers with single order)
monetary_features["std_order_value"] = monetary_features["std_order_value"].fillna(0)

monetary_features.head()


,Customer ID,total_revenue,avg_order_value,max_order_value,min_order_value,std_order_value,total_items,revenue_per_item
0,12346.0,1.04,1.040000,1.04,1.04,0.000000,74215,0.000014
1,12347.0,481.21,2.644011,12.75,0.25,2.255381,2458,0.195773
2,12348.0,178.71,5.764839,40.00,0.29,13.400323,2341,0.076339
3,12349.0,605.10,8.289041,300.00,0.42,35.028021,631,0.958954
4,12350.0,65.30,3.841176,40.00,0.85,9.334751,197,0.331472


In [57]:
rfm = rfm.merge(monetary_features.drop(columns=["total_items"]), 
                on="Customer ID", how="left")


In [58]:
rfm.shape
rfm.head()


,Customer ID,Recency,Frequency,Monetary,Churn,avg_quantity_per_order,max_quantity,min_quantity,std_quantity,total_items_purchased,unique_products,unique_invoices,total_revenue,avg_order_value,max_order_value,min_order_value,std_order_value,revenue_per_item
0,12346.0,326,1,1.04,1,74215.000000,74215,74215,0.000000,74215,1,1,1.04,1.040000,1.04,1.04,0.000000,0.000014
1,12347.0,2,7,481.21,0,13.505495,240,2,18.856172,2458,103,7,481.21,2.644011,12.75,0.25,2.255381,0.195773
2,12348.0,75,4,178.71,0,75.516129,144,1,51.091990,2341,22,4,178.71,5.764839,40.00,0.29,13.400323,0.076339
3,12349.0,19,1,605.10,0,8.643836,36,1,6.982856,631,73,1,605.10,8.289041,300.00,0.42,35.028021,0.958954
4,12350.0,310,1,65.30,1,11.588235,24,1,4.345383,197,17,1,65.30,3.841176,40.00,0.85,9.334751,0.331472


- customer_tenure_days -	How long customer stayed active
- active_days - Engagement intensity
- active_months -	Consistency over months
- avg_days_between_orders - Purchase regularity
- purchase_span_days - Lifecycle length

In [69]:
# Ensure InvoiceDate is datetime
clean_df["InvoiceDate"] = pd.to_datetime(clean_df["InvoiceDate"])

temporal_features = clean_df.groupby("Customer ID").agg(
    first_purchase_date=("InvoiceDate", "min"),
    last_purchase_date=("InvoiceDate", "max"),
    active_days=("InvoiceDate", "nunique"),
    active_months=("InvoiceDate", lambda x: x.dt.to_period("M").nunique()),
    total_orders=("Invoice", "nunique")
).reset_index()

# Reference date already defined earlier
# reference_date = clean_df["InvoiceDate"].max() + pd.Timedelta(days=1)

# Temporal calculations
temporal_features["customer_tenure_days"] = (
    temporal_features["last_purchase_date"] - temporal_features["first_purchase_date"]
).dt.days

temporal_features["days_since_first_purchase"] = (
    reference_date - temporal_features["first_purchase_date"]
).dt.days

temporal_features["purchase_span_days"] = (
    temporal_features["last_purchase_date"] - temporal_features["first_purchase_date"]
).dt.days

# Average gap between orders
temporal_features["avg_days_between_orders"] = (
    temporal_features["purchase_span_days"] /
    (temporal_features["total_orders"] - 1)
)

# Handle customers with only 1 order
temporal_features["avg_days_between_orders"] = (
    temporal_features["avg_days_between_orders"].fillna(0)
)

temporal_features.head()


,Customer ID,first_purchase_date,last_purchase_date,active_days,active_months,total_orders,customer_tenure_days,days_since_first_purchase,purchase_span_days,avg_days_between_orders
0,12346.0,2011-01-18 10:01:00,2011-01-18 10:01:00,1,1,1,0,326,0,0.000000
1,12347.0,2010-12-07 14:57:00,2011-12-07 15:52:00,7,7,7,365,367,365,60.833333
2,12348.0,2010-12-16 19:09:00,2011-09-25 13:13:00,4,4,4,282,358,282,94.000000
3,12349.0,2011-11-21 09:51:00,2011-11-21 09:51:00,1,1,1,0,19,0,0.000000
4,12350.0,2011-02-02 16:01:00,2011-02-02 16:01:00,1,1,1,0,310,0,0.000000


In [77]:
rfm = rfm.merge(
    temporal_features.drop(columns=[
        "first_purchase_date",
        "last_purchase_date",
        "total_orders"
    ]),
    on="Customer ID",
    how="left"
)


In [81]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4339 entries, 0 to 4338
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Customer ID                4339 non-null   float64
 1   Recency                    4339 non-null   int64  
 2   Frequency                  4339 non-null   int64  
 3   Monetary                   4339 non-null   float64
 4   Churn                      4339 non-null   int64  
 5   avg_quantity_per_order     4339 non-null   float64
 6   max_quantity               4339 non-null   int64  
 7   min_quantity               4339 non-null   int64  
 8   std_quantity               4339 non-null   float64
 9   total_items_purchased      4339 non-null   int64  
 10  unique_products            4339 non-null   int64  
 11  unique_invoices            4339 non-null   int64  
 12  total_revenue              4339 non-null   float64
 13  avg_order_value            4339 non-null   float

In [80]:
# Feature 1: Order consistency
rfm["order_consistency"] = (
    rfm["Frequency"] / rfm["customer_tenure_days"].replace(0, 1)
)

# Feature 2: Spend consistency
rfm["spend_consistency"] = (
    rfm["avg_order_value"] / (rfm["std_order_value"] + 1)
)

# Quick check
rfm[["order_consistency", "spend_consistency"]].describe()


,order_consistency,spend_consistency
count,4339.000000,4339.000000
mean,0.413351,1.716091
std,0.726058,32.824092
min,0.005479,0.000000
25%,0.020408,0.672443
50%,0.046512,0.787493
75%,1.000000,0.924593
max,34.000000,2033.100000


In [83]:
rfm.to_csv("C:/Users/DELL/Desktop/GPP_task/ecommerce-churn-prediction/data/processed/customer_features_final.csv", index=False)

In [85]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392733 entries, 0 to 541909
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Invoice         392733 non-null  object        
 1   StockCode       392733 non-null  object        
 2   Description     392733 non-null  object        
 3   Quantity        392733 non-null  int64         
 4   InvoiceDate     392733 non-null  datetime64[ns]
 5   Price           392733 non-null  float64       
 6   Customer ID     392733 non-null  float64       
 7   Country         392733 non-null  object        
 8   financial_year  392733 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 30.0+ MB


In [86]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4339 entries, 0 to 4338
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Customer ID                4339 non-null   float64
 1   Recency                    4339 non-null   int64  
 2   Frequency                  4339 non-null   int64  
 3   Monetary                   4339 non-null   float64
 4   Churn                      4339 non-null   int64  
 5   avg_quantity_per_order     4339 non-null   float64
 6   max_quantity               4339 non-null   int64  
 7   min_quantity               4339 non-null   int64  
 8   std_quantity               4339 non-null   float64
 9   total_items_purchased      4339 non-null   int64  
 10  unique_products            4339 non-null   int64  
 11  unique_invoices            4339 non-null   int64  
 12  total_revenue              4339 non-null   float64
 13  avg_order_value            4339 non-null   float

## Modeling

In [90]:
from sklearn.model_selection import train_test_split

# Separate features and target
X = rfm.drop(columns=["Customer ID", "Churn"])
y = rfm["Churn"]

X.shape, y.shape


((4339, 24), (4339,))

In [91]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


((3471, 24), (868, 24))

In [92]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [93]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    random_state=42
)

log_model.fit(X_train_scaled, y_train)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",42
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:

In [94]:
from sklearn.metrics import roc_auc_score

y_train_proba = log_model.predict_proba(X_train_scaled)[:, 1]
train_auc = roc_auc_score(y_train, y_train_proba)

train_auc


0.999990670277145

In [95]:
y_test_proba = log_model.predict_proba(X_test_scaled)[:, 1]
test_auc = roc_auc_score(y_test, y_test_proba)

test_auc


0.9998090919937955

In [96]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=10,
    random_state=42,
    class_weight="balanced"
)

rf_model.fit(X_train, y_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",10
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",10
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric(

In [97]:
rf_test_proba = rf_model.predict_proba(X_test)[:, 1]
rf_test_auc = roc_auc_score(y_test, rf_test_proba)

rf_test_auc


1.0

In [99]:
import joblib
from pathlib import Path

MODEL_DIR = Path("C:/Users/DELL/Desktop/GPP_task/ecommerce-churn-prediction/models")
MODEL_DIR.mkdir(exist_ok=True)

joblib.dump(log_model, MODEL_DIR / "logistic_model.pkl")
joblib.dump(rf_model, MODEL_DIR / "random_forest_model.pkl")
joblib.dump(scaler, MODEL_DIR / "scaler.pkl")


['C:\\Users\\DELL\\Desktop\\GPP_task\\ecommerce-churn-prediction\\models\\scaler.pkl']

In [103]:
def run_pipeline():
    # ALL your existing code stays here
    pass


if __name__ == "__main__":
    run_pipeline()
